In [32]:
import pandas as pd
import random
data=pd.read_csv('../data/regular_season_results.csv')
season=data.groupby(['season'])
#print(season.get_group('A').head())
data=season.get_group('A')
tourney=pd.read_csv('../data/tourney_results.csv')
#print(tourney.head())
#print(np.array(data['lteam']).shape)
#print(np.array(tourney['lteam']).shape)
input1=zip(data['wteam'],data['lteam'])
def data_formatter(game):
    tmp=[0,1]
    random.shuffle(tmp)
    if tmp.index(0)==1:
        return (game[tmp[0]],game[tmp[1]],0)
    else:
        return (game[tmp[0]],game[tmp[1]],1)
    
inputs=map(lambda x: data_formatter(x),input1)
team1=np.array(map(lambda x: x[0],inputs))
team2=np.array(map(lambda x: x[1],inputs))
results=np.array(map(lambda x: x[2],inputs))
print(results)

[0 0 1 ..., 0 1 0]


In [51]:
import pandas as pd
import keras
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.preprocessing.text import one_hot
from keras.layers import merge,Reshape, Flatten
from keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np

#data=pd.read_csv('../data/regular_season_results.csv')
data['wloc']=data['wloc'].map({'H':1,'A':0})
teams = pd.read_csv('../data/teams.csv')
nteams=len(teams['id'])
win=pd.get_dummies(data['wteam'])
lose=pd.get_dummies(data['lteam'])
input_data=np.array(win + lose)

input_1 = Input(batch_shape=(None,1),dtype='float32',name='w_team')
input_2 = Input(batch_shape=(None,1),dtype='float32',name='l_team')

#shared embedding layer
emb = Embedding(output_dim=64,input_dim=857,input_length=1)

emb1 = emb(input_1)
emb2 = emb(input_2)
emba = Reshape((64,))(emb1)
embb = Reshape((64,))(emb2)

game = merge([emb1,emb2],mode='concat',concat_axis=1)
flatten = Reshape((128,))(game)
game = Dense(64, activation='relu')(flatten)
#needs to be a custom layer??
perf1 = merge([emba,game], mode = 'dot',dot_axes=(1,1))
perf2 = merge([embb,game], mode = 'dot',dot_axes=(1,1))
perf = merge([perf1,perf2],mode= lambda x: x[0]-x[1],output_shape=(1,))
print(K.int_shape(perf))
pred=Activation('sigmoid')(perf)
print(K.int_shape(pred))

#flatten = Reshape((128,))(game)
#game = Dense(64, activation='relu')(flatten)
#pred = Dense(1,activation='sigmoid')(game)
model=Model(input=(input_1,input_2),output=pred)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.fit([team1,team2],results,batch_size=32)
tourney=tourney.groupby(['season']).get_group('A')
model.test_on_batch([np.array(tourney['wteam']),np.array(tourney['lteam'])],np.ones(len(tourney['wteam'])))
#print(classification_report(np.ones(len(preds)),np.rint(preds)))
#print(confusion_matrix(np.ones(len(preds)),np.rint(preds)))
#percentage correct
#print(np.sum(preds)/63.0)

(None, None)
(None, None)
Epoch 1/10
4123/4123 [==============================] - 0s - loss: 0.6838 - acc: 0.6299     
Epoch 2/10
4123/4123 [==============================] - 0s - loss: 0.6031 - acc: 0.7120     
Epoch 3/10
4123/4123 [==============================] - 0s - loss: 0.5352 - acc: 0.7256     
Epoch 4/10
4123/4123 [==============================] - 0s - loss: 0.5207 - acc: 0.7342     
Epoch 5/10
4123/4123 [==============================] - 0s - loss: 0.5144 - acc: 0.7388     
Epoch 6/10
4123/4123 [==============================] - 0s - loss: 0.5102 - acc: 0.7451     
Epoch 7/10
4123/4123 [==============================] - 0s - loss: 0.5065 - acc: 0.7441     
Epoch 8/10
4123/4123 [==============================] - 0s - loss: 0.5037 - acc: 0.7472     
Epoch 9/10
4123/4123 [==============================] - 0s - loss: 0.5016 - acc: 0.7469     
Epoch 10/10
4123/4123 [==============================] - 0s - loss: 0.4995 - acc: 0.7471     


[0.50490326, 0.76089698]

In [53]:
from keras.layers.core import Flatten
data=pd.read_csv('../data/regular_season_results.csv')
data['wloc']=data['wloc'].map({'H':1,'A':0})
teams = pd.read_csv('../data/teams.csv')
nteams=len(teams['id'])
win=pd.get_dummies(data['wteam'])
lose=-1.0*pd.get_dummies(data['lteam'])
input_data=np.array(win + lose)

#Implementation of Terry-Bradley ANN

model=Sequential()
model.add(Dense(1,input_dim=nteams))
model.add(Activation('sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(input_data,[1 for i in range(0,len(input_data))])

#model.predict()

Epoch 1/10
91224/91224 [==============================] - 4s - loss: 0.2620 - acc: 0.9868     
Epoch 2/10
91224/91224 [==============================] - 4s - loss: 0.0178 - acc: 1.0000     
Epoch 3/10
91224/91224 [==============================] - 4s - loss: 0.0011 - acc: 1.0000     
Epoch 4/10
91224/91224 [==============================] - 5s - loss: 6.2915e-05 - acc: 1.0000     
Epoch 5/10
91224/91224 [==============================] - 5s - loss: 3.7673e-06 - acc: 1.0000     
Epoch 6/10
91224/91224 [==============================] - 5s - loss: 2.5382e-07 - acc: 1.0000     
Epoch 7/10
91224/91224 [==============================] - 5s - loss: 1.1923e-07 - acc: 1.0000     
Epoch 8/10
91224/91224 [==============================] - 5s - loss: 1.1921e-07 - acc: 1.0000     
Epoch 9/10
91224/91224 [==============================] - 5s - loss: 1.1921e-07 - acc: 1.0000     
Epoch 10/10
91224/91224 [==============================] - 5s - loss: 1.1921e-07 - acc: 1.0000     


In [57]:
print(input_data[0].shape)
print(pd.get_dummies(tourney['wteam']).shape)

(356,)
(63, 32)


In [3]:
# rankings looks reasonable 

weights=model.layers[0].get_weights()
    
print(weights[0].shape)
ranks=zip(teams['name'],map(lambda x:x[0],weights[0]))
ranks=sorted(ranks,key=lambda x: x[1],reverse=True)
print(ranks)

(356, 1)
[('Duke', 1.5817878), ('Kansas', 1.5185366), ('Kentucky', 1.2558224), ('North Carolina', 1.1737632), ('Syracuse', 1.1448064), ('Connecticut', 1.1141794), ('Arizona', 1.0802886), ('Michigan St', 1.0385598), ('Florida', 1.0196065), ('Gonzaga', 1.0105392), ('Texas', 0.9768309), ('Louisville', 0.97504449), ('Cincinnati', 0.97021908), ('Memphis', 0.95779139), ('Wisconsin', 0.9278518), ('Illinois', 0.90887874), ('Pittsburgh', 0.90706897), ('Maryland', 0.90286481), ('Oklahoma', 0.8937549), ('UCLA', 0.8909415), ('Ohio St', 0.88501954), ('Oklahoma St', 0.87141299), ('Stanford', 0.86533934), ('Villanova', 0.86345059), ('Georgetown', 0.84876126), ('Xavier', 0.83903414), ('Marquette', 0.81679571), ('Tennessee', 0.81606609), ('Missouri', 0.76348424), ('New Mexico', 0.73766464), ('Notre Dame', 0.71736729), ('Wake Forest', 0.71504498), ('Indiana', 0.71109766), ('Purdue', 0.70197904), ('Creighton', 0.69647616), ('California', 0.66604292), ('NC State', 0.66436505), ('UNLV', 0.66119802), ('Butl